In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50

In [2]:
def create_resnet50_embedding_model():
    # Load ResNet50 model without the top classification layers
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # Add a Global Average Pooling layer to get embeddings
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    embedding_model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return embedding_model

def preprocess_images(images):
    preprocessed_images = []
    for img in images:
        img_resized = tf.image.resize(img, (224, 224))  # Resize to 224x224
        img_array = img_to_array(img_resized)
        img_array = np.array(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
        preprocessed_images.append(img_array[0])
    return np.array(preprocessed_images)


# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # Randomly select 10,000 samples from the training set
np.random.seed(42)
num_samples = 10000
random_indices = np.random.choice(x_train.shape[0], num_samples, replace=False)
x_train = x_train[random_indices]
y_train = y_train[random_indices]

np.random.seed(42)
num_samples = 2000
random_indices = np.random.choice(x_test.shape[0], num_samples, replace=False)
x_test = x_test[random_indices]
y_test = y_test[random_indices]


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [3]:
x_train.shape

(10000, 32, 32, 3)

In [5]:
embedding_model = create_resnet50_embedding_model()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [6]:
x_train_preprocessed = preprocess_images(x_train)
x_train_embeddings = embedding_model.predict(x_train_preprocessed)

313/313 ━━━━━━━━━━━━━━━━━━━━ 306s 974ms/step


In [7]:
x_train_embeddings.shape

(10000, 2048)

In [8]:
x_train_embeddings[0].shape

(2048,)

In [9]:
x_test_preprocessed = preprocess_images(x_test)
x_test_embeddings = embedding_model.predict(x_test_preprocessed)

63/63 ━━━━━━━━━━━━━━━━━━━━ 61s 960ms/step


In [10]:
x_test_embeddings.shape

(2000, 2048)

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
np.savez('/content/drive/My Drive/Colab Notebooks/DIP_proj/resnet50_embeddings_x_train.npz', embeddings=x_train_embeddings, labels=y_train)
np.savez('/content/drive/My Drive/Colab Notebooks/DIP_proj/resnet50_embeddings_x_test.npz', embeddings=x_test_embeddings, labels=y_test)
